In [37]:
import os
import h5py
from matplotlib import pyplot as plt
from medpy.metric import binary
from calibrate.evaluation import metrics
import pandas as pd
import numpy as np
from skimage.transform import resize
import glob

In [38]:
def metric_multi_class(gt, pred):
    
    try:
        cnums = np.unique(gt)

        dsc_cls = []
        hd_cls = []

        for cno in cnums:

            dsc_cls.append(round(binary.dc(pred == cno, gt == cno),4))
            hd_cls.append(round(binary.hd(pred == cno , gt == cno),4))
        dsc_m, hd_m = np.round(np.mean(dsc_cls),4), np.round(np.mean(hd_cls),4)
        
    except Exception as e:
        dsc_m, hd_m = 1, 0
        
    return dsc_m, hd_m
    

In [55]:
def metric_dsc(gt, pred):
    
    try:
        cnums = np.unique(gt)

        dsc_cls = []

        for cno in cnums:

            dsc_cls.append(round(binary.dc(pred == cno, gt == cno),4))
        dsc_m = np.round(np.mean(dsc_cls),4)
        
    except Exception as e:
        print (e)
        dsc_m = 1
        
    return dsc_m
    

In [39]:
model_names = ['CE','CD','FL','CP','LS','VS','ML']

## mrbrains

In [23]:
model_dirs = ['/home/ar88770/MarginLoss/outputs/brainatlas/unet-ce-adam/20220404-16:20:24-829672',
              '/home/ar88770/MarginLoss/outputs/brainatlas/unet-ce_dice-adam/20220427-17:25:09-405389',
              '/home/ar88770/MarginLoss/outputs/brainatlas/unet-focal-adam/20220406-10:59:04-103885',
              '/home/ar88770/MarginLoss/outputs/brainatlas/unet-penalty_ent-adam/20220406-11:26:33-769991',
              '/home/ar88770/MarginLoss/outputs/brainatlas/unet-ls-adam/20220404-16:49:47-579491',
              '/home/ar88770/MarginLoss/outputs/brainatlas/unet-svls-adam/20220417-17:23:24-173305',
              '/home/ar88770/MarginLoss/outputs/brainatlas/unet-logit_margin-adam/20220404-22:15:12-835794']
gt_dir = '/home/ar88770/MarginLoss/mrbrains/valid/'
gt_files = glob.glob(gt_dir + '*.h5')

In [30]:
metric_dict_dsc = {'fname':[],'CE':[],'CD':[],'FL':[],'CP':[],'LS':[],'VS':[],'ML':[]}
metric_dict_hd = {'fname':[],'CE':[],'CD':[],'FL':[],'CP':[],'LS':[],'VS':[],'ML':[]}

for gt_path in gt_files:
    
    fname = os.path.basename(gt_path)
    metric_dict_dsc['fname'].append(fname)
    metric_dict_hd['fname'].append(fname)
    
    with h5py.File(gt_path, 'r') as hf:
        gt = hf['mask'][:]
    
    for ii, model_dir in enumerate(model_dirs):
        
        res_path = os.path.join(model_dir, 'results', fname)
        
        with h5py.File(res_path, 'r') as hf:
            pred = hf['mask'][:][8:-8,8:-8]
            
        dsc, hd = metric_multi_class(pred, gt)
        metric_dict_dsc[model_names[ii]].append(dsc)
        metric_dict_hd[model_names[ii]].append(hd)

df_dsc = pd.DataFrame(metric_dict_dsc)
df_hd = pd.DataFrame(metric_dict_hd)

In [32]:
df_dsc.mean(), df_hd.mean()

/tmp/ipykernel_26352/3876129717.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_dsc.mean(), df_hd.mean()


(CE    0.83660
 CD    0.81930
 FL    0.81275
 CP    0.83720
 LS    0.80635
 VS    0.80420
 ML    0.85580
 dtype: float64,
 CE    13.08075
 CD    13.32975
 FL    13.98160
 CP    14.84150
 LS    17.81365
 VS    13.20420
 ML    13.99945
 dtype: float64)

## abdomen

In [34]:
model_dirs = ['/home/ar88770/MarginLoss/outputs/abdomen/unet-ce-adam/20220501-23:43:11-336368',
              '/home/ar88770/MarginLoss/outputs/abdomen/unet-ce_dice-adam/20220502-18:52:50-928615',
              '/home/ar88770/MarginLoss/outputs/abdomen/unet-focal-adam/20220502-09:49:29-657707',
              '/home/ar88770/MarginLoss/outputs/abdomen/unet-penalty_ent-adam/20220502-12:21:41-348902',
              '/home/ar88770/MarginLoss/outputs/abdomen/unet-ls-adam/20220502-04:02:05-859276',
              '/home/ar88770/MarginLoss/outputs/abdomen/unet-svls-adam/20220502-15:09:37-710891',
              '/home/ar88770/MarginLoss/outputs/abdomen/unet-logit_margin-adam/20220502-07:01:38-850966']
gt_dir = '/home/ar88770/MarginLoss/flare/valid/'
gt_files = glob.glob(gt_dir + '*.h5')

In [46]:
metric_dict_dsc = {'fname':[],'CE':[],'CD':[],'FL':[],'CP':[],'LS':[],'VS':[],'ML':[]}
# metric_dict_hd = {'fname':[],'CE':[],'CD':[],'FL':[],'CP':[],'LS':[],'VS':[],'ML':[]}

for gt_path in gt_files:
    
    fname = os.path.basename(gt_path)
    metric_dict_dsc['fname'].append(fname)
    metric_dict_hd['fname'].append(fname)
    
    with h5py.File(gt_path, 'r') as hf:
        gt = hf['mask'][:]
    
    for ii, model_dir in enumerate(model_dirs):
        
        res_path = os.path.join(model_dir, 'results', fname)
        
        with h5py.File(res_path, 'r') as hf:
            pred = hf['mask'][:]
            
        dsc = metric_dsc(pred, gt)
        metric_dict_dsc[model_names[ii]].append(dsc)
        # metric_dict_hd[model_names[ii]].append(hd)

df_dsc = pd.DataFrame(metric_dict_dsc)
# df_hd = pd.DataFrame(metric_dict_hd)

## promise

In [50]:
model_dirs = ['/home/ar88770/MarginLoss/outputs/prostate_mc/unet-ce-adam/20220409-22:47:25-847975',
              '/home/ar88770/MarginLoss/outputs/prostate_mc/unet-ce_dice-adam/20220427-13:38:39-924288',
              '/home/ar88770/MarginLoss/outputs/prostate_mc/unet-focal-adam/20220406-17:06:46-112063',
              '/home/ar88770/MarginLoss/outputs/prostate_mc/unet-penalty_ent-adam/20220411-23:10:20-338365',
              '/home/ar88770/MarginLoss/outputs/prostate_mc/unet-ls-adam/20220409-22:57:05-534513',
              '/home/ar88770/MarginLoss/outputs/prostate_mc/unet-svls-adam/20220417-14:10:04-419455',
              '/home/ar88770/MarginLoss/outputs/prostate_mc/unet-logit_margin-adam/20220409-23:07:04-480575']
gt_dir = '/home/ar88770/MarginLoss/promise_mc/valid/'
gt_files = glob.glob(gt_dir + '*.h5')

In [58]:
metric_dict_dsc = {'fname':[],'CE':[],'CD':[],'FL':[],'CP':[],'LS':[],'VS':[],'ML':[]}
# metric_dict_hd = {'fname':[],'CE':[],'CD':[],'FL':[],'CP':[],'LS':[],'VS':[],'ML':[]}

for gt_path in gt_files:
    
    fname = os.path.basename(gt_path)
    metric_dict_dsc['fname'].append(fname)
    metric_dict_hd['fname'].append(fname)
    
    with h5py.File(gt_path, 'r') as hf:
        gt = hf['mask'][:][:,80:240,80:240]
    
    for ii, model_dir in enumerate(model_dirs):
        
        res_path = os.path.join(model_dir, 'results', fname)
        
        with h5py.File(res_path, 'r') as hf:
            pred = hf['mask'][:]
            
        dsc = metric_dsc(pred, gt)
        metric_dict_dsc[model_names[ii]].append(dsc)
        # metric_dict_hd[model_names[ii]].append(hd)

df_dsc = pd.DataFrame(metric_dict_dsc)
# df_hd = pd.DataFrame(metric_dict_hd)

In [61]:
df_dsc.mean()

/tmp/ipykernel_26352/3044381647.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_dsc.mean()


CE    0.656143
CD    0.667729
FL    0.473543
CP    0.669529
LS    0.678429
VS    0.680643
ML    0.698157
dtype: float64

## brats

In [62]:
model_dirs = ['/home/ar88770/MarginLoss/outputs/brain/unet-ce-adam/20220327-22:46:38-903888',
              '/home/ar88770/MarginLoss/outputs/brain/unet-ce_dice-adam/20220427-15:57:36-359885',
              '/home/ar88770/MarginLoss/outputs/brain/unet-focal-adam/20220405-15:44:14-601824',
              '/home/ar88770/MarginLoss/outputs/brain/unet-penalty_ent-adam/20220405-23:10:20-562585',
              '/home/ar88770/MarginLoss/outputs/brain/unet-ls-adam/20220328-02:43:30-442461',
              '/home/ar88770/MarginLoss/outputs/brain/unet-svls-adam/20220417-14:12:05-491765',
              '/home/ar88770/MarginLoss/outputs/brain/unet-logit_margin-adam/20220328-06:40:26-544931']
gt_dir = '/home/ar88770/MarginLoss/brats/valid/'
gt_files = glob.glob(gt_dir + '*.h5')

In [ ]:
metric_dict_dsc = {'fname':[],'CE':[],'CD':[],'FL':[],'CP':[],'LS':[],'VS':[],'ML':[]}
# metric_dict_hd = {'fname':[],'CE':[],'CD':[],'FL':[],'CP':[],'LS':[],'VS':[],'ML':[]}

for gt_path in gt_files:
    
    fname = os.path.basename(gt_path)
    metric_dict_dsc['fname'].append(fname)
    metric_dict_hd['fname'].append(fname)
    
    with h5py.File(gt_path, 'r') as hf:
        gt = hf['mask'][:]
    
    for ii, model_dir in enumerate(model_dirs):
        
        res_path = os.path.join(model_dir, 'results', fname)
        
        with h5py.File(res_path, 'r') as hf:
            pred = hf['mask'][:][:,8:-8,8:-8]
            
        dsc = metric_dsc(pred, gt)
        metric_dict_dsc[model_names[ii]].append(dsc)
        # metric_dict_hd[model_names[ii]].append(hd)

df_dsc = pd.DataFrame(metric_dict_dsc)
# df_hd = pd.DataFrame(metric_dict_hd)

In [64]:
df_dsc.mean()

/tmp/ipykernel_26352/3044381647.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_dsc.mean()


CE    0.748027
CD    0.710202
FL    0.708940
CP    0.730351
LS    0.728979
VS    0.756600
ML    0.729373
dtype: float64